In [25]:
%reset -f
%config InteractiveShell.ast_node_interactivity = 'all'

# Fetching all Air Quality datasets into their dataframes
# Perform immediate concatenation per year
import os
import pandas as pd

# Initialize
dir = 'CAData/'
colidx = [0,2,4,5,17]     # column indexes to use
innerkeys = ['Date', 'Site ID', 'COUNTY']
dataA2020 = pd.read_csv(dir + 'cf-2020-co.csv', parse_dates=True, usecols=colidx)
dataA2021 = pd.read_csv(dir + 'cf-2021-co.csv', parse_dates=True, usecols=colidx)
dataA2022 = pd.read_csv(dir + 'cf-2022-co.csv', parse_dates=True, usecols=colidx)

with os.scandir(dir) as datasets:
    for dataset in datasets:
        if dataset.is_file() and 'co' not in dataset.name:
            temp = pd.read_csv(dataset, parse_dates=True, usecols=colidx)
            if '2020' in dataset.name:
                # cols_to_use = dataA2020.columns.difference(temp)
                dataA2020 = pd.merge(dataA2020, temp, how='inner', on=innerkeys)
            elif '2021' in dataset.name:
                # cols_to_use = dataA2021.columns.difference(temp)
                dataA2021 = pd.merge(dataA2021, temp, how='inner', on=innerkeys)
            elif '2022' in dataset.name:
                # cols_to_use = dataA2022.columns.difference(temp)
                dataA2022 = pd.merge(dataA2022, temp, how='inner', on=innerkeys)

dataA2020.groupby(by=['Date', 'Site ID']).mean().groupby(by=['Date']).mean()
dataA2021.groupby(by=['Date', 'Site ID']).mean().groupby(by=['Date']).mean()
dataA2022.groupby(by=['Date', 'Site ID']).mean().groupby(by=['Date']).mean()

C:\Users\YSHebron\AppData\Local\Temp\ipykernel_22628\2669712743.py:23: FutureWarning: Passing 'suffixes' which cause duplicate columns {'UNITS_x'} in the result is deprecated and will raise a MergeError in a future version.
  dataA2020 = pd.merge(dataA2020, temp, how='inner', on=innerkeys)
C:\Users\YSHebron\AppData\Local\Temp\ipykernel_22628\2669712743.py:23: FutureWarning: Passing 'suffixes' which cause duplicate columns {'UNITS_x'} in the result is deprecated and will raise a MergeError in a future version.
  dataA2020 = pd.merge(dataA2020, temp, how='inner', on=innerkeys)
C:\Users\YSHebron\AppData\Local\Temp\ipykernel_22628\2669712743.py:26: FutureWarning: Passing 'suffixes' which cause duplicate columns {'UNITS_x'} in the result is deprecated and will raise a MergeError in a future version.
  dataA2021 = pd.merge(dataA2021, temp, how='inner', on=innerkeys)
C:\Users\YSHebron\AppData\Local\Temp\ipykernel_22628\2669712743.py:26: FutureWarning: Passing 'suffixes' which cause duplicate 

,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
01/04/2020,1.170000,31.460000,0.030200,0.008908,38.000000,19.685000,0.740000
01/10/2020,0.750000,33.075000,0.020500,0.006750,29.000000,12.605000,0.800000
01/16/2020,1.000000,36.583333,0.041000,0.010053,57.333333,24.605556,2.700000
01/22/2020,0.400000,29.250000,0.036000,0.003000,27.500000,15.450000,0.500000
01/28/2020,0.625000,34.562500,0.032500,0.004996,38.500000,9.706250,0.875000
01/31/2020,0.800000,28.000000,0.037000,0.006850,23.000000,7.100000,0.400000
02/03/2020,0.150000,7.000000,0.040000,0.004000,40.500000,3.350000,0.200000
02/09/2020,0.380000,16.400000,0.037600,0.002867,21.800000,7.905000,0.320000
02/15/2020,0.700000,34.300000,0.052000,0.006000,49.000000,18.575000,0.900000


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
01/04/2021,0.825,32.875,0.027000,0.006750,34.250000,23.492500,0.75
01/10/2021,0.850,37.250,0.035000,0.004500,23.500000,7.675000,0.60
01/16/2021,1.150,46.425,0.034500,0.021000,36.500000,10.372500,1.20
01/22/2021,0.750,33.175,0.038000,0.004500,23.750000,7.712500,0.75
01/28/2021,0.700,33.875,0.036500,0.007250,16.250000,6.997500,0.90
...,...,...,...,...,...,...,...
12/06/2021,1.350,50.100,0.032667,0.017100,75.833333,35.493333,1.70
12/12/2021,0.730,24.800,0.035400,0.004442,21.900000,11.703000,0.46
12/18/2021,0.775,36.175,0.030500,0.004000,18.000000,7.042500,0.55


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
01/05/2022,0.920000,35.900000,0.026200,0.006173,39.700000,17.716000,0.840000
01/11/2022,0.525000,42.800000,0.035500,0.004500,20.583333,3.985000,0.700000
01/17/2022,1.080000,33.230000,0.026400,0.009557,46.033333,20.744333,1.680000
01/23/2022,0.525000,30.450000,0.037500,0.004000,14.500000,5.077500,1.250000
01/29/2022,0.680000,35.760000,0.029800,0.010918,31.900000,12.767000,0.660000
...,...,...,...,...,...,...,...
12/07/2022,0.710000,27.950000,0.029800,0.005165,18.600000,10.765000,0.480000
12/13/2022,0.783333,29.716667,0.028667,0.005527,25.555556,11.185000,0.733333
12/19/2022,0.775000,32.483333,0.025667,0.006849,35.916667,20.600556,0.600000
